In [30]:
class GameOptions:
  winningScore = 1000
  rollsPerTurn = 3
  trackSize = 10

  def __repr__(self):
    return str(self.__dict__)

  def __hash__(self):
    return hash(repr(self))

  def __eq__(self, other):
    if not isinstance(other, type(self)):
      return False
    return self.winningScore == other.winningScore\
      and self.rollsPerTurn == other.rollsPerTurn\
      and self.trackSize == other.trackSize

    
class Player:
  score = 0

  def __init__(self, startingPosition):
    self.trackPosition = startingPosition

  def __repr__(self):
    return str(self.__dict__)

  def __hash__(self):
    return hash(repr(self))
    
  def __eq__(self, other):
    if not isinstance(other, type(self)):
      return False
    return self.score == other.score and self.trackPosition == other.trackPosition

class DeterministicDice:
  value = 0
  rolls = 0

  def __init__(self, sides):
    self.sides = sides
    
  def __repr__(self):
    return str(self.__dict__)

  def roll(self):
    self.value += 1
    self.rolls += 1
    result = self.value
    self.value %= self.sides
    return result

In [31]:
def simulateGame(startingPositions, rollDice, options=None):
  def createPlayers():
    for start in startingPositions:
      yield Player(start)

  if options == None:
    options = GameOptions()

  players = tuple(createPlayers())
  currentPlayer = 0
  
  while True:
    player = players[currentPlayer]
    rolls = []
    for _ in range(options.rollsPerTurn):
      rolls.append(rollDice())
    
    player.trackPosition += sum(rolls) 
    player.trackPosition %= options.trackSize
    player.score += player.trackPosition + 1

    if player.score >= options.winningScore:
      return players

    currentPlayer += 1
    currentPlayer %= len(players)

In [32]:
import itertools
import functools
import copy

@functools.cache
def getPossbileRollValues(rolls):
  if rolls == 0:
    return [0]
  else:
    values = getPossbileRollValues(rolls - 1)
    results = []
    for rollValue in values:
      for newRoll in range(1, 4):
        results.append(rollValue + newRoll)
    return results
      
@functools.cache
def getPossibilities(players, gameOptions, playerIndex=0):
  def addResults(newResults):
    for i, result in enumerate(newResults):
      results[i] += result

  results = list(itertools.repeat(0, len(players)))
  for rollSum in getPossbileRollValues(gameOptions.rollsPerTurn):
    player = copy.copy(players[playerIndex])
    player.trackPosition += rollSum
    player.trackPosition %= gameOptions.trackSize
    player.score += player.trackPosition + 1

    if player.score >= gameOptions.winningScore:
      results[playerIndex] += 1
    else:
      newPlayers = list(players)
      newPlayers[playerIndex] = player
      newPlayerIndex = playerIndex + 1
      newPlayerIndex %= len(players)
      addResults(getPossibilities(tuple(newPlayers), gameOptions, newPlayerIndex))
  
  return results

In [33]:
dice = DeterministicDice(100)
players = simulateGame([7, 2], dice.roll)
dice.rolls * min(map(lambda x: x.score, players))

412344

In [34]:
gameOptions = GameOptions()
gameOptions.winningScore = 21
players = (Player(7), Player(2))
getPossibilities.args = set()
print(getPossibilities(players, gameOptions))
print(getPossibilities.cache_info())


[214924284932572, 143154512703677]
CacheInfo(hits=397436, misses=26878, maxsize=None, currsize=26878)
